# init all requirements

In [1]:
import torch
import torch.nn as nn


# Параметры
lr = 0.3
batch_size = 32
in_ch = 1024
out_ch = 608

# Матрицы и вектораы для ручного линейного слоя
Y_gt = torch.randn(batch_size, out_ch)

X = torch.randn(batch_size, in_ch)
W = torch.randn(out_ch, in_ch)
b = torch.randn(1, out_ch)

# Линейный слой для сравнения
linear = nn.Linear(in_ch, out_ch)
with torch.no_grad():
    linear.weight.copy_(W.clone())
    linear.bias.copy_(b.squeeze().clone())

optimizer = torch.optim.SGD(linear.parameters(), lr=lr)

X_t = X.clone()
Y_pred = linear(X_t)

loss = ((Y_pred - Y_gt) ** 2).mean()
loss.backward()

# Градиенты для проверки
dW_torch = linear.weight.grad.clone()
db_torch = linear.bias.grad.clone()

optimizer.step()

# regular linear

In [2]:
'''
Обычная реализация прямого и обратного прохода линейного слоя, и корректировки весов:
'''

Y = X @ W.T + b

# По аналогии с тем, как получаются градиенты MSE, 
# чтобы было совпадение с линейным слоем torch-а
dY = 2 * (Y - Y_gt) / (X.shape[0] * out_ch) 

# в текущем пайплайне вычислять dX особо смысла нет,
# т.к. блок один и градиент передавать не надо
dX = dY @ W
dW = X.T @ dY
db = dY.sum(dim=0)

print("dX: ", dX.shape, ", dW: ", dW.shape, ", db: ", db.shape)

W_manual = W - lr * dW.T
b_manual = b - lr * db.unsqueeze(dim=0)

# ПРоверям, что вручную написанный линейный слой 
# ведет себя идентично torch.nn.Linear-у
print(f"\nIs weights matching nn.Linear: {torch.allclose(W_manual, linear.weight, atol=1e-6)}")
print(f"Is bias matching nn.Linear:    {torch.allclose(b_manual, linear.bias.unsqueeze(0), atol=1e-6)}")
print(f"Is grad W matching nn.Linear:  {torch.allclose(dW.T, dW_torch, atol=1e-6)}")
print(f"Is grad b matching nn.Linear:  {torch.allclose(db.squeeze(), db_torch, atol=1e-6)}")

print("\n\n\n\n\n")

dX:  torch.Size([32, 1024]) , dW:  torch.Size([1024, 608]) , db:  torch.Size([608])

Is weights matching nn.Linear: True
Is bias matching nn.Linear:    True
Is grad W matching nn.Linear:  True
Is grad b matching nn.Linear:  True








# column-wise tensor parallel


![column_wise_tp_scheme.png](https://nanotron-ultrascale-playbook.static.hf.space/assets/images/tp_diagram2.png)

In [3]:
'''
Идейная реализация column-wise tensor_parallel:
'''


# Разбиваем на два устройства (на shard-ы)
part_size = W.shape[0] // 2

W_gpu1 = W[:part_size, :].clone()
W_gpu2 = W[part_size:, :].clone()
b_gpu1 = b[:, :part_size].clone()
b_gpu2 = b[:, part_size:].clone()

# broadcast
X_gpu1 = X.clone()
X_gpu2 = X.clone()

print("Input shapes:")
print("W_gpu1: ", W_gpu1.shape, 
      "\nW_gpu2: ", W_gpu2.shape, 
      "\nb_gpu1: ", b_gpu1.shape, 
      "\nb_gpu2: ", b_gpu2.shape, 
      "\nX_gpu1: ", X_gpu1.shape, 
      "\nX_gpu2: ", X_gpu2.shape,
     )


# ------------------------ #
# Основной пайплайн прямого и обратного прохода, внутри которого 
# не должна создаваться полная матрица весов W

# ----FORWARD----
# считаем части предиктов для каждой части (на каждом device-е)
Y_gpu1 = X_gpu1 @ W_gpu1.T + b_gpu1
Y_gpu2 = X_gpu2 @ W_gpu2.T + b_gpu2

# all_gather
Y = torch.cat((Y_gpu1, Y_gpu2), 1)


# ----BACKWARD----
dY = 2 * (Y - Y_gt) / (X.shape[0] * out_ch) 

# Cчитаем градиенты для shard-а на каждой gpu
dX_gpu1 = dY[:, :part_size] @ W_gpu1
dX_gpu2 = dY[:, part_size:] @ W_gpu2
# в текущем пайплайне вычислять dX особо смысла нет,
# т.к. блок один и градиент передавать не надо
dX = dX_gpu1 + dX_gpu2

dW_gpu1 = X_gpu1.T @ dY[:, :part_size]
dW_gpu2 = X_gpu2.T @ dY[:, part_size:]
db_gpu1 = dY[:, :part_size].sum(dim=0)
db_gpu2 = dY[:, part_size:].sum(dim=0)

# Обновление весов
W_gpu1 -= lr * dW_gpu1.T
W_gpu2 -= lr * dW_gpu2.T
b_gpu1 -= lr * db_gpu1.unsqueeze(dim=0)
b_gpu2 -= lr * db_gpu2.unsqueeze(dim=0)
# ------------------------ #


# Объединяем матрица и градиенты для сравнения
W_comb = torch.cat((W_gpu1, W_gpu2), 0)
b_comb = torch.cat((b_gpu1, b_gpu2), 1)
dW_comb = torch.cat((dW_gpu1, dW_gpu2), 1)
db_comb = torch.cat((db_gpu1, db_gpu2), 0)

print("\nW_comb: ", W_comb.shape, 
      "\nb_comb: ", b_comb.shape, 
      "\ndW_comb: ", dW_comb.shape, 
      "\ndb_comb: ", db_comb.shape,
     )

# Проверяем, идентичен ли пайплайн nn.Linear-у
print(f"\nIs weights matching nn.Linear: {torch.allclose(W_comb, linear.weight, atol=1e-6)}")
print(f"Is bias matching nn.Linear:    {torch.allclose(b_comb, linear.bias.unsqueeze(0), atol=1e-6)}")
print(f"Is grad W matching nn.Linear:  {torch.allclose(dW_comb.T, dW_torch, atol=1e-6)}")
print(f"Is grad b matching nn.Linear:  {torch.allclose(db_comb, db_torch, atol=1e-6)}")

print("\n\n\n\n\n")

Input shapes:
W_gpu1:  torch.Size([304, 1024]) 
W_gpu2:  torch.Size([304, 1024]) 
b_gpu1:  torch.Size([1, 304]) 
b_gpu2:  torch.Size([1, 304]) 
X_gpu1:  torch.Size([32, 1024]) 
X_gpu2:  torch.Size([32, 1024])

W_comb:  torch.Size([608, 1024]) 
b_comb:  torch.Size([1, 608]) 
dW_comb:  torch.Size([1024, 608]) 
db_comb:  torch.Size([608])

Is weights matching nn.Linear: True
Is bias matching nn.Linear:    True
Is grad W matching nn.Linear:  True
Is grad b matching nn.Linear:  True








# row-wise tensor parallel


![row_wise_tp_scheme.png](https://nanotron-ultrascale-playbook.static.hf.space/assets/images/tp_diagram3.png)

In [4]:
'''
Идейная реализация row-wise tensor_parallel:
'''


# Разбиваем на два устройства (на shard-ы)
part_size = W.shape[1] // 2

W_gpu1 = W[:, :part_size].clone()
W_gpu2 = W[:, part_size:].clone()

b_gpu1 = b.clone()
b_gpu2 = b.clone()

# scatter
X_gpu1 = X[:, :part_size]
X_gpu2 = X[:, part_size:]

print("Input shapes:")
print("W_gpu1: ", W_gpu1.shape, 
      "\nW_gpu2: ", W_gpu2.shape, 
      "\nb_gpu1: ", b_gpu1.shape, 
      "\nb_gpu2: ", b_gpu2.shape, 
      "\nX_gpu1: ", X_gpu1.shape, 
      "\nX_gpu2: ", X_gpu2.shape,
     )

# ------------------------ #
# Основной пайплайн прямого и обратного прохода, внутри которого 
# не должна создаваться полная матрица весов W

# ----FORWARD----
# считаем части предиктов для каждой части (на каждом device-е)
Y_gpu1 = X_gpu1 @ W_gpu1.T
Y_gpu2 = X_gpu2 @ W_gpu2.T

Y = Y_gpu1 + Y_gpu2 + b

dY = 2 * (Y - Y_gt) / (X.shape[0] * out_ch) 

dX_gpu1 = dY @ W_gpu1
dW_gpu1 = dY.T @ X_gpu1

dX_gpu2 = dY @ W_gpu2
dW_gpu2 = dY.T @ X_gpu2

# в текущем пайплайне вычислять dX особо смысла нет,
# т.к. блок один и градиент передавать не надо
dX = torch.cat((dX_gpu1, dX_gpu2), 1)
db = dY.sum(dim=0)

W_gpu1 -= lr * dW_gpu1
W_gpu2 -= lr * dW_gpu2

b_tp_row = b - lr * db.unsqueeze(dim=0)


# Объединяем матрица и градиенты для сравнения
W_comb = torch.cat((W_gpu1, W_gpu2), 1)
dW_comb = torch.cat((dW_gpu1, dW_gpu2), 1)

print("\nW_comb: ", W_comb.shape, 
      "\nb_comb: ", b_tp_row.shape, 
      "\ndW_comb: ", dW_comb.shape, 
      "\ndb_comb: ", db.shape,
      "\ndX: ", dX.shape,
     )

# Проверяем, идентичен ли пайплайн nn.Linear-у
print(f"\nIs weights matching nn.Linear: {torch.allclose(W_comb, linear.weight, atol=1e-6)}")
print(f"Is bias matching nn.Linear:    {torch.allclose(b_tp_row, linear.bias.unsqueeze(0), atol=1e-6)}")
# Пришлось добавить atol=1e-6
# (подробнее описание ниже и в readme)
dW_diff = (dW_comb - dW_torch).abs().max()
print(f"Is grad W matching nn.Linear:  {torch.allclose(dW_comb, dW_torch, atol=1e-6)} (max diff = {dW_diff})")
print(f"Is grad b matching nn.Linear:  {torch.allclose(db, db_torch, atol=1e-6)}")

print("\n\n\n\n\n")

Input shapes:
W_gpu1:  torch.Size([608, 512]) 
W_gpu2:  torch.Size([608, 512]) 
b_gpu1:  torch.Size([1, 608]) 
b_gpu2:  torch.Size([1, 608]) 
X_gpu1:  torch.Size([32, 512]) 
X_gpu2:  torch.Size([32, 512])

W_comb:  torch.Size([608, 1024]) 
b_comb:  torch.Size([1, 608]) 
dW_comb:  torch.Size([608, 1024]) 
db_comb:  torch.Size([608]) 
dX:  torch.Size([32, 1024])

Is weights matching nn.Linear: True
Is bias matching nn.Linear:    True
Is grad W matching nn.Linear:  True (max diff = 4.0978193283081055e-08)
Is grad b matching nn.Linear:  True








# Вывод:

Пункты задания выполнены. Пункт о том, что "Ни в какой момент времени полная матрица ни на одном из устройств  
не должна появляться в памяти" строго выполняются внутри Forward и Backward. Поскольку оценка и инициализация находятся  
вне **column-wise** и **row-wise tensor_parallel (TP)**, можно считать условие выполненным.  

Реализовать **column-wise TP** было довольно просто, как и реализовать **row-wise TP**.

Основные проблемы возникли при сравнении градиентов и весов с таргетами, полученными из **torch.nn.Linear**,  
через **torch.allclose()**.  
Тогда как итоговый **dW** у ручной реализации и **column-wise TP** были идентичными матрицам из torch.allclose(),  
значения **dW** получаемого в ходе работы **row-wise TP** незначительно отличались от целевых, что при использовании  
torch.allclose() без atol приводило возникновению False при сравнении.  

С увеличением размерности эта проблема проявляется и в остальных двух пайплайнах.

После подсчета фактического diff-а между row-wise **dW** и таргетным **dW** было обнаружено несоответствие на **e-08** доли.  
Как я понимаю, проблема возникает из-за того, что операции FP32 неассоциативны (описано в статье https://arxiv.org/html/2506.09501v1).  
А обнаружил я ее на более низких размерностях именно у **row-wise TP** потому, что в отличе от предыдущих двух способов у row-wise  
GEMM разбит на два независимых, это отличие от оригинального nn.Linear приводит к изменению порядка FP суммирования и как следствие к  
более выраженному численному расхождению.

![row_wise_tp_scheme.png](https://github.com/VilisovEvgeny/LinearTensorParallel/blob/main/images/fp32_tablet.png?raw=true)

Поскольку данная проблема является скорее фундаментальной особенностью, пока что, с ней остается только смириться.  
Поэтому применяем atol=1e-6 и проходим проверку, игнорируя незначительные отличия.